In [55]:
import pandas as pd
import geopandas as gp
import numpy as np
from datetime import date

In [56]:
## initiate file paths
sample_outing_name = 's4_s6'

In [57]:
# create output file names
input_results_path = f"/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/{sample_outing_name}_results_joined_SL_{date.today().strftime('%Y%m%d')}.csv"
sample_pts_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/Lookup Tables/Sampling_Sites_MASTER.xlsx"
output_results_path = f"{sample_outing_name}_results_join_geometry_{date.today().strftime('%Y%m%d')}"

In [58]:
sample_pts = pd.read_excel(sample_pts_path, sheet_name = "Sample_Locations")
sample_pts_gdf = gp.GeoDataFrame(sample_pts, geometry=gp.points_from_xy(sample_pts.Longitude, sample_pts.Latitude), crs="EPSG:4326")

In [59]:
screening_results = pd.read_csv(input_results_path)

# TODO: remove any screening results with U and C qualifiers. do not want to show these in the map

In [60]:
try:
    screening_results['DATE'] = screening_results['DATE'].str[:10]
    screening_results["DATE"] = pd.to_datetime(screening_results["DATE"])
except:
    print("date not included")

In [61]:
sample_pts_gdf["Date"] = pd.to_datetime(sample_pts_gdf["Date"], format='mixed')

In [65]:
sample_pts_join_results = pd.merge(screening_results, sample_pts_gdf, left_on = ['Sample ID', 'DATE'], right_on = ['Sampling ID', 'Date'], how = 'left')
sample_pts_join_results.rename(columns = {'Medium_x':'Medium'}, inplace = True)

In [63]:
sample_pts_join_results_gdf = gp.GeoDataFrame(sample_pts_join_results)
sample_pts_join_results_gdf['Date'] = sample_pts_join_results_gdf['Date'].astype(str)

In [64]:
sample_pts_join_results[['DATE', 'Sample ID', 'Medium', 'Chemical Group', 'Chemical', 'Scenario',
       'Screening Level Type', 'Screening Level Measurement', 'SL Unit',
       'Source', 'Parameter in Spreadsheet', 'Result Value',
       'Result Value Units', 'SL_exceeded', 'SL_diff','Latitude', 'Longitude','Description']].to_csv(f"/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/{output_results_path}.csv", index = False)